In [3]:
import sys
import pandas as pd
from pathlib import Path

SRC_DIR = Path("..") / "src"
sys.path.append(str(SRC_DIR))

pd.set_option("display.max_columns", None)

DATA_DIR = Path("../data")

# Core tables
recipes = pd.read_csv(DATA_DIR / "recipes.csv")
recipe_feedback = pd.read_csv(DATA_DIR / "recipe_feedback.csv")

# Output from pantry matching
ingredients = pd.read_csv(DATA_DIR / "ingredients.csv")
recipe_ingredients = pd.read_csv(DATA_DIR / "recipe_ingredients.csv")
pantry = pd.read_csv(DATA_DIR / "pantry.csv")

In [ ]:
from matcher import (
    compute_recipe_ingredient_status,
    compute_recipe_match_metrics,
    get_missing_ingredients
)

ingredient_status = compute_recipe_ingredient_status(
    recipe_ingredients,
    pantry
)

recipe_metrics = compute_recipe_match_metrics(ingredient_status)
missing_ingredients = get_missing_ingredients(
    ingredient_status,
    ingredients
)

In [5]:
feedback_agg = (
    recipe_feedback
    .groupby("recipe_id")
    .agg(
        avg_rating=("rating", "mean"),
        times_cooked=("rating", "count"),
        would_make_again_rate=("would_make_again", "mean")
    )
    .reset_index()
)

display(feedback_agg)

,recipe_id,avg_rating,times_cooked,would_make_again_rate
0,1,4.5,1,1.0
1,2,2.0,1,0.0


In [6]:
# Merge Everything Into One Scoring Table
scoring_df = (
    recipe_metrics
    .merge(recipes, on="recipe_id", how="left")
    .merge(feedback_agg, on="recipe_id", how="left")
)

# Fill missing feedback with neutral defaults
scoring_df["avg_rating"] = scoring_df["avg_rating"].fillna(3.0)
scoring_df["times_cooked"] = scoring_df["times_cooked"].fillna(0)
scoring_df["would_make_again_rate"] = scoring_df["would_make_again_rate"].fillna(0.5)

display(scoring_df)

,recipe_id,total_ingredients,available_count,missing_count,partial_count,pantry_match_pct,name,cuisine,dish_type,requires_airfryer,requires_soaking,meal_prep_type,video_url,avg_rating,times_cooked,would_make_again_rate
0,1,5,2,3,0,40.0,Vegetable Upma,Indian,breakfast,False,False,good_for_2_days,https://youtube.com/xxx,4.5,1.0,1.0
1,2,1,1,0,0,100.0,Chana Salad,Indian,salad,False,True,prep_components,https://youtube.com/yyy,2.0,1.0,0.0
2,4,2,2,0,0,100.0,Overnight Oats,Global,breakfast,False,True,good_for_3_days,https://youtube.com/aaa,3.0,0.0,0.5


In [7]:
def apply_constraints(df, allow_airfryer=True, allow_soaking=True):
    filtered = df.copy()
    
    if not allow_airfryer:
        filtered = filtered[filtered["requires_airfryer"] == False]
    
    if not allow_soaking:
        filtered = filtered[filtered["requires_soaking"] == False]
    
    return filtered

In [8]:
scoring_df = apply_constraints(
    scoring_df,
    allow_airfryer=False,
    allow_soaking=False
)

In [9]:
# Normalizing scores

scoring_df["pantry_score"] = scoring_df["pantry_match_pct"] / 100
scoring_df["rating_score"] = scoring_df["avg_rating"] / 5
scoring_df["repeat_score"] = scoring_df["would_make_again_rate"]

In [10]:
scoring_df["final_score"] = (
    0.6 * scoring_df["pantry_score"] +
    0.3 * scoring_df["rating_score"] +
    0.1 * scoring_df["repeat_score"]
)

In [11]:
scoring_df.loc[scoring_df["avg_rating"] < 2.5, "final_score"] *= 0.3
scoring_df.loc[scoring_df["would_make_again_rate"] < 0.3, "final_score"] *= 0.5

In [12]:
ranked_recipes = scoring_df.sort_values(
    by=["final_score", "pantry_match_pct"],
    ascending=False
)

display(
    ranked_recipes[
        ["name", "dish_type", "pantry_match_pct",
         "avg_rating", "times_cooked", "final_score"]
    ]
)

,name,dish_type,pantry_match_pct,avg_rating,times_cooked,final_score
0,Vegetable Upma,breakfast,40.0,4.5,1.0,0.61


In [13]:
def explain_recipe(recipe_name):
    row = ranked_recipes[ranked_recipes["name"] == recipe_name].iloc[0]
    
    explanation = {
        "Pantry Match %": row["pantry_match_pct"],
        "Avg Rating": row["avg_rating"],
        "Would Make Again %": row["would_make_again_rate"],
        "Final Score": row["final_score"]
    }
    
    return explanation

explain_recipe("Vegetable Upma")

{'Pantry Match %': 40.0,
 'Avg Rating': 4.5,
 'Would Make Again %': 1.0,
 'Final Score': 0.61}